**Sentimental Analysis using Naive Bayes(94% Accuracy) and Support Vector Machine(98% Accuracy)**

In [2]:
#Import all neccessary libraries
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk

In [3]:
#read the data file
Corpus = pd.read_csv('sentiment_tweets3.csv')

In [4]:
Corpus.head()

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [5]:
# Check if there is a null value
Corpus['message to examine'].isnull().sum()

0

In [6]:
#Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['message to examine'] = [entry.lower() for entry in Corpus['message to examine']]

In [7]:
#Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['message to examine']= [word_tokenize(entry) for entry in Corpus['message to examine']]

In [8]:
Corpus['message to examine']

0        [just, had, a, real, good, moment, ., i, misss...
1        [is, reading, manga, http, :, //plurk.com/p/mz...
2        [@, comeagainjen, http, :, //twitpic.com/2y2lx...
3        [@, lapcat, need, to, send, 'em, to, my, accou...
4        [add, me, on, myspace, !, !, !, myspace.com/lo...
                               ...                        
10309    [no, depression, by, g, herbo, is, my, mood, f...
10310    [what, do, you, do, when, depression, succumbs...
10311    [ketamine, nasal, spray, shows, promise, again...
10312    [dont, mistake, a, bad, day, with, depression,...
10313                                                  [0]
Name: message to examine, Length: 10314, dtype: object

In [9]:
#Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting
#We need to tag words with noun, verb or adjective to pass to WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
# nltk.download('wordnet')
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [10]:
for index,entry in enumerate(Corpus['message to examine']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [12]:
#lets see the data
Corpus['text_final']

0               ['real', 'good', 'moment', 'miss', 'much']
1                                ['read', 'manga', 'http']
2                         ['comeagainjen', 'http', 'http']
3        ['lapcat', 'need', 'send', 'accountant', 'tomo...
4                                       ['add', 'myspace']
                               ...                        
10309    ['depression', 'g', 'herbo', 'mood', 'do', 'st...
10310    ['depression', 'succumb', 'brain', 'make', 'fe...
10311    ['ketamine', 'nasal', 'spray', 'show', 'promis...
10312    ['dont', 'mistake', 'bad', 'day', 'depression'...
10313                                                   []
Name: text_final, Length: 10314, dtype: object


In [13]:
#Data looks good now, lets do the Train and Test split
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label (depression result)'],test_size=0.3)

In [14]:
#I am using TFidf to create the vectors of the input, we can also use bag of words or anything else to see the accuracy
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [15]:
print(Tfidf_vect.vocabulary_)
print(Train_X_Tfidf)

{'real': 3565, 'good': 1823, 'moment': 2854, 'miss': 2826, 'much': 2896, 'read': 3560, 'http': 2099, 'comeagainjen': 864, 'need': 2951, 'send': 3863, 'tomorrow': 4495, 'even': 1460, 'refer': 3601, 'tax': 4349, 'support': 4277, 'evidence': 1473, 'though': 4435, 'add': 42, 'myspace': 2913, 'sleepy': 4007, 'time': 4467, 'tonight': 4499, 'someone': 4063, 'already': 132, 'say': 3798, 'home': 2053, 'mean': 2723, 'least': 2492, 'regular': 3616, 'possible': 3355, 'today': 4484, 'nice': 2983, 'nite': 3001, 'twitterville': 4607, 'workout': 4903, 'night': 2989, 'sweet': 4303, 'dream': 1294, 'morning': 2874, 'everybody': 1467, 'finally': 1602, 'create': 996, 'wordpress': 4898, 'blog': 503, 'seattle': 3837, 'coffee': 840, 'community': 877, 'cnt': 832, 'get': 1777, 'til': 4464, 'frm': 1710, 'remember': 3640, 'ur': 4665, 'top': 4508, 'nicolerichie': 2987, 'yes': 4966, 'band': 349, 'awesome': 302, 'please': 3314, 'reply': 3653, 'really': 3571, 'love': 2602, 'reflection': 3604, 'shadow': 3899, 'ooo': 3

In [16]:
#Fir the model using NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  93.24717285945073


In [17]:
#Using SVM
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  98.96607431340873


In [18]:
#We can clearly see that this model fit better using SVM with accuracy around 99%